# Lab 04. 파이토치를 사용하여 소프트맥스 회귀 모델을 학습하고 예측하는 실습
---

- Iris 데이터셋을 사용하여 진행한다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

## 데이터 로드 

In [2]:
iris = load_iris()
# print(iris)

x = iris.data
y = iris.target
# print(x,y)

x_train, x_test , y_train, y_test = train_test_split(x,y, test_size=0.1, random_state=777)

# convert data to Pytorch tensor 
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

## 데이터셋 데이터 로더 정의 

In [3]:
train_dataset = TensorDataset(x_train, y_train) 
# TensorDataset() -> 동일한 크기를 가진 텐서들을 첫번째 차원을 기준으로 결합해서 데이터셋 생성
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

## 모델 구현 

In [4]:
class SoftmaxRegression(nn.Module) : 
    def __init__(self, input_size, num_classes) : 
        super(SoftmaxRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, x) : 
        out = self.linear(x)
        
        return out

## 하이퍼 파라미터 설정 

In [5]:
input_size = 4 
num_classes = 3 
lr = 0.01 
num_epochs = 100

## 옵티마이저 loss 함수 모델 호출 

In [6]:
model = SoftmaxRegression(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

## 모델 훈련

In [7]:
total_step = len(train_loader) 
for epoch in range(num_epochs) : 
    for i, (inputs, labels) in enumerate(train_loader) : 
        # Forward pass 
        outputs= model(inputs)
        outputs = outputs.float()
        loss = criterion(outputs, labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # print
        if (i+1) % 10 == 0 :
            print("Epoch [{}/{}] , Step [{}/{}], Loss : {:.4f}".format(
                epoch+1, num_epochs, i+1, total_step, loss.item()
            ))

Epoch [1/100] , Step [10/14], Loss : 0.8975
Epoch [2/100] , Step [10/14], Loss : 0.8869
Epoch [3/100] , Step [10/14], Loss : 0.8699
Epoch [4/100] , Step [10/14], Loss : 0.7805
Epoch [5/100] , Step [10/14], Loss : 0.8376
Epoch [6/100] , Step [10/14], Loss : 0.7486
Epoch [7/100] , Step [10/14], Loss : 0.6207
Epoch [8/100] , Step [10/14], Loss : 0.5639
Epoch [9/100] , Step [10/14], Loss : 0.6946
Epoch [10/100] , Step [10/14], Loss : 0.4399
Epoch [11/100] , Step [10/14], Loss : 0.6605
Epoch [12/100] , Step [10/14], Loss : 0.6199
Epoch [13/100] , Step [10/14], Loss : 0.5700
Epoch [14/100] , Step [10/14], Loss : 0.5749
Epoch [15/100] , Step [10/14], Loss : 0.6136
Epoch [16/100] , Step [10/14], Loss : 0.4011
Epoch [17/100] , Step [10/14], Loss : 0.3882
Epoch [18/100] , Step [10/14], Loss : 0.5983
Epoch [19/100] , Step [10/14], Loss : 0.5462
Epoch [20/100] , Step [10/14], Loss : 0.4117
Epoch [21/100] , Step [10/14], Loss : 0.5067
Epoch [22/100] , Step [10/14], Loss : 0.5013
Epoch [23/100] , St

In [8]:
model.eval()
with torch.no_grad() : 
    outputs = model(x_test) 
    _, predicted = torch.max(outputs.data, 1)
    acc = (predicted == y_test).sum().item() / len(y_test)
    print("Test ACC >> {:.2f}%".format(acc * 100))

Test ACC >> 100.00%
